## **Segmenting and Clustering Neighborhoods in Toronto**

### Introduction

This notebook will be used to explore, segment, and cluster the neighborhoods in the city of Toronto. 

### Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

1. <a href="#1">Scrape Web Data & Transform into Pandas Dataframe</a> 
<br> 
2. <a href="#2">Get Neighborhood Geographical Coordinates</a>
<br>    
3. <a href="#3">Explore and Cluster Neighborhoods</a>

</font>
</div>

Importing all necessary modules:

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment if library not installed
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

<a id='1'></a>

## 1. Scrape Web Data & Transform into Pandas Dataframe
First, the dataset will be created by scraping a Wikipedia page that contains information about Toronto neighborhoods. The data will be scraped, cleaned, and read into a Pandas dataframe.

In [5]:
# Web scraping using wikipedia library to access and parse data from Wikipedia
!pip install wikipedia
import wikipedia as wp

# Get html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

# Read html into Pandas dataframe
toronto_df = pd.DataFrame(data=df)
toronto_df.head()

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


*Clean dataframe:*

In [7]:
# rename columns
toronto_df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True) 
# remove rows with no assigned boroughs
toronto_df.drop(toronto_df[toronto_df['Borough']=='Not assigned'].index, inplace = True)
# combine neighborhoods in the same postal code area (separated by a comma)
toronto_df = toronto_df.groupby('PostalCode').agg(lambda x : ','.join(set(x))).reset_index()
# for rows with a borough but a Not assigned neighborhood, make neighborhood the same as the borough
toronto_df['Neighborhood'].replace('Not assigned', toronto_df['Borough'], inplace=True) 

print(toronto_df.shape)
toronto_df.head(15)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [201]:
# The code was removed by Watson Studio for sharing.

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<a id='2'></a>

## 2. Get Neighborhood Geographical Coordinates

Next, the latitude and longitude coordinates of each neighborhood will be added to the dataframe.

In [8]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
!pip install geocoder
import geocoder # import geocoder

     |████████████████████████████████| 102kB 13.5MB/s ta 0:00:01


In [14]:
toronto_df.PostalCode.head()

0    M1B
1    M1C
2    M1E
3    M1G
4    M1H
Name: PostalCode, dtype: object

In [35]:
postal_codes = toronto_df['PostalCode'].tolist()
latitude = []
longitude = []

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    for code in postal_codes:
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
        latitude.append(g.latlng[0])
        longitude.append(g.latlng[1])

In [36]:
toronto_df['Latitude'] = latitude
toronto_df['Longitude'] = longitude
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",43.726276,-79.263625
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.713054,-79.285055
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside",43.724235,-79.227925
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.696770,-79.259967


<a id='3'></a>

## 3. Explore and Cluster Neighborhoods
Now that the dataframe is complete, the Toronto neighborhoods will be explored through segmentation and clustering.

##### Create a map of Toronto with neighborhoods superimposed on top:

In [37]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [38]:
# Define Foursquare API credentials and version

CLIENT_ID = 'DAKCHZI5XKGCONNVDSCZIROAQXWJ0UBSWHBLO02WSK04OBYC' # your Foursquare ID
CLIENT_SECRET = 'QA5OHTIDKPBOI3EDRQYT211KF1N53SC33XW0YKFBXPTMQKWA' # your Foursquare Secret
VERSION = '20191010' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DAKCHZI5XKGCONNVDSCZIROAQXWJ0UBSWHBLO02WSK04OBYC
CLIENT_SECRET:QA5OHTIDKPBOI3EDRQYT211KF1N53SC33XW0YKFBXPTMQKWA


### Explore Toronto neighborhoods:

In [90]:
# create new dataframe with only neighborhoods within a Toronto Borough

toronto_data = toronto_df[toronto_df['Borough'].str.contains('Toronto')]
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295425
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667
43,M4M,East Toronto,Studio District,43.660629,-79.334855
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133


In [92]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Explore Nearby Venues in Each Neighborhood:

In [46]:
# create a function to get nearby venues in each neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [94]:
# run above function on each neighborhood and create a new dataframe called toronto_venues
limit = 100 # limit 100 venues
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Summerhill West,South Hill,Deer Park,Forest Hill SE,Rathnelly
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Regent Park,Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond,Adelaide,King
Harbourfront East,Union Station,Toronto Islands
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill West,Forest Hill North
Yorkville,North Midtown,The Annex
Harbord,University of Toronto
Grange Park,Chinatown,Kensington Market
King and Spadina,South Niagara,Railway Lands,Harbourfront West,CN Tower,Island airport,Bathurst Quay
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Exhibition Place,Brockton,Parkdale Village
The Junction South,High Park
Roncesvalles,Parkda

In [95]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()

(1769, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,65,65,65,65,65,65
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,100,100,100,100,100,100
Christie,9,9,9,9,9,9
Church and Wellesley,82,82,82,82,82,82
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26
Davisville North,6,6,6,6,6,6


In [96]:
# Analyse Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [172]:
# group rows by neighborhood and take the mean of the frequency of occurrence of each category (to be used for clustering)

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.015385,0.00,0.000000,0.000000,0.000000,0.015385,0.000000,0.015385,0.030769,0.000000,0.000000,0.015385,0.030769,0.000000,0.015385,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030769,0.00,0.000000,0.000000,0.030769,0.000000,0.000000,0.00,0.000000,0.015385,0.046154,0.076923,0.000000,0.000000,0.00,0.00,0.015385,0.00,0.015385,0.000000,0.015385,0.000000,0.015385,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.00000,0.015385,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030769,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.00,0.015385,0.015385,0.015385,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.01538

##### Create a new dataframe and display the top 10 venues for each neighborhood:

In [173]:
print(toronto_grouped.shape)

# create function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

(37, 225)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Seafood Restaurant,Breakfast Spot,Steakhouse,Hotel,Beer Bar,Café
1,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Steakhouse,Hotel,Bar,Pizza Place,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Seafood Restaurant
2,"Cabbagetown,St. James Town",Pizza Place,Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Market,Pharmacy,Chinese Restaurant
3,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Italian Restaurant,Middle Eastern Restaurant,Bakery,Spa,Chinese Restaurant,Ice Cream Shop,Plaza
4,Christie,Café,Grocery Store,Playground,Italian Restaurant,Baby Store,Coffee Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Fish Market


##### Cluster Neighborhoods wth K-means clustering:

In [238]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4], dtype=int32)

Add clusters and top 10 venues to a new dataframe:

In [239]:
# add clustering labels
toronto_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how = 'right')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676531,-79.295425,4,Health Food Store,Pub,Trail,Yoga Studio,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105,0,Bus Line,Park,Discount Store,Grocery Store,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667,4,Burger Joint,Italian Restaurant,Fast Food Restaurant,Burrito Place,Park,Liquor Store,Steakhouse,Sandwich Place,Sushi Restaurant,Movie Theater
43,M4M,East Toronto,Studio District,43.660629,-79.334855,4,Diner,Italian Restaurant,Coffee Shop,Pizza Place,Brewery,Bar,Café,Sushi Restaurant,Arts & Crafts Store,Music Store
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,3,Bus Line,Swim School,Lawyer,Yoga Studio,Event Space,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


##### Visualize Clusters:

In [240]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Explore Clusters:

In [241]:
toronto_merged['Cluster Labels'].value_counts()

4    28
0     5
2     2
3     1
1     1
Name: Cluster Labels, dtype: int64

### Cluster 1:

In [243]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Bus Line,Park,Discount Store,Grocery Store,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
46,Central Toronto,0,Playground,Park,Gym Pool,Garden,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
50,Downtown Toronto,0,Playground,Gym / Fitness Center,Park,Candy Store,Grocery Store,Building,Health & Beauty Service,Eastern European Restaurant,Fish Market,Fish & Chips Shop
59,Downtown Toronto,0,Pier,Harbor / Marina,Park,Yoga Studio,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
75,Downtown Toronto,0,Café,Grocery Store,Playground,Italian Restaurant,Baby Store,Coffee Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Fish Market


****Cluster 1 top venues = outdoors & recreation (parks, playgrounds, gym)***

### Cluster 2:

In [193]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Gym,Playground,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


****Cluster 2 top venues = recreation (gym, playgrounds)***

### Cluster 3:

In [194]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,2,Home Service,Park,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
82,West Toronto,2,Convenience Store,Park,Sandwich Place,Yoga Studio,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


****Cluster 2 top venues = professional & other places (home service & convenience store)***

### Cluster 4:

In [195]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Bus Line,Swim School,Lawyer,Yoga Studio,Event Space,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


****Cluster 4 top venues = bus, swim school, lawyer***

### Cluster 5:

In [196]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Health Food Store,Pub,Trail,Yoga Studio,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
42,East Toronto,4,Burger Joint,Italian Restaurant,Fast Food Restaurant,Burrito Place,Park,Liquor Store,Steakhouse,Sandwich Place,Sushi Restaurant,Movie Theater
43,East Toronto,4,Diner,Italian Restaurant,Coffee Shop,Pizza Place,Brewery,Bar,Café,Sushi Restaurant,Arts & Crafts Store,Music Store
45,Central Toronto,4,Food & Drink Shop,Breakfast Spot,Gym,Park,Clothing Store,Hotel,Flea Market,Flower Shop,Food,Ethiopian Restaurant
47,Central Toronto,4,Dessert Shop,Coffee Shop,Thai Restaurant,Café,Sandwich Place,Pizza Place,Italian Restaurant,Farmers Market,Park,Seafood Restaurant
49,Central Toronto,4,Coffee Shop,Light Rail Station,Supermarket,Liquor Store,Yoga Studio,Event Space,Food,Flower Shop,Flea Market,Fish Market
51,Downtown Toronto,4,Pizza Place,Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Market,Pharmacy,Chinese Restaurant
52,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Burger Joint,Bubble Tea Shop,Pub,Dance Studio,Men's Store
53,Downtown Toronto,4,Coffee Shop,Bakery,Theater,Café,Boat or Ferry,Health & Beauty Service,Italian Restaurant,Breakfast Spot,Spanish Restaurant,Spa
54,Downtown Toronto,4,Coffee Shop,Clothing Store,Cosmetics Shop,Italian Restaurant,Middle Eastern Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Hotel,Bookstore


****Cluster 5 top venues = food & drink (coffee shops, cafes, etc)***